<a href="https://colab.research.google.com/github/wall456/Taylor_bot/blob/main/Taylor_Swift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U transformers datasets sentencepiece pandas scikit-learn

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.6.0)
Requirement already up-to-date: datasets in /usr/local/lib/python3.7/dist-packages (1.6.2)
Requirement already up-to-date: sentencepiece in /usr/local/lib/python3.7/dist-packages (0.1.95)
Requirement already up-to-date: pandas in /usr/local/lib/python3.7/dist-packages (1.2.4)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pathlib, shutil, contextlib, torch, numpy as np, pandas as pd
from typing import Any
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
 
class NLP:
    def __init__(self,
        base = AutoModelForSequenceClassification,
        checkpoint = 'distilbert-base-uncased',
        task = 'text-generation',
        max_seq_length = 128,
        padding_side = 'left',
        path = '/content/drive/MyDrive/NLP/tswift',
        path_model = None,
        overwrite = False,
        kwargs = dict(),
        ):
        
        self.base = base
        self.task = task
        self.checkpoint = checkpoint
        self.max_seq_length = max_seq_length
        self.path = pathlib.Path(path)
        if path_model:
            self.path_model = pathlib.Path(path_model)
        else:
            self.path_model = self.path / f'{task}-{checkpoint.split("/")[-1]}'
        self.path_model.mkdir(parents=True, exist_ok=True)
        if overwrite:
            shutil.rmtree(self.path_model, ignore_errors=True)

        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True,
            model_max_length=self.max_seq_length, padding_side=padding_side)
        if self.tokenizer._pad_token is None:
            self.tokenizer._pad_token = self.tokenizer._eos_token

        with contextlib.suppress(OSError):
            self.model = base.from_pretrained(self.path_model)
            self.trained = True
            print(f'Using existing model at {str(self.path_model)}')

        if not hasattr(self, 'model'):
            print(f'No model found at {str(self.path_model)} - creating new one')
            self.model = base.from_pretrained(checkpoint, max_length=self.max_seq_length,
                task_specific_params={'text-generation':{'do_sample':True, 'max_length': self.max_seq_length}},
                **kwargs)
            self.trained = False

    def __getitem__(self, key):
        return getattr(self, key)
 
    def __setitem__(self, key, value):
        return setattr(self, key, value)
 
    def _listify(self, x):
        if isinstance(x, list) or isinstance(x, tuple) or isinstance(x, range):
            return list(x)
        else:
            return [x]

    def splitter(self, ds, frac=0.2):
        frac = min(frac, 1-frac)
        idxes = next(StratifiedShuffleSplit(test_size=frac).split(ds['labels'], ds['labels']))
        return [ds.select(idx) for idx in idxes]

    def make_dataset(self,
        source_data,
        text_col      = 'text',
        label_col     = 'label',
        holdout_frac  = 0.1,
        validate_frac = 0.2,
        ):

        if not self.trained:
            self.df = pd.DataFrame(source_data).dropna(subset=[text_col, label_col]).sample(frac=1.0).reset_index(drop=True)
            tokens = dict()
            for label, text in self.df[[label_col, text_col]].values:
                tensors = self.preprocess(text)['wrapped']
                tensors['labels'] = [label for _ in range(tensors['input_ids'].shape[0])]
                for key, val in tensors.items():
                    try:
                        tokens[key].append(val)
                    except:
                        tokens[key] = [val]
            L = np.hstack(tokens.pop('labels'))
            id2label = dict(enumerate(np.unique(L)))
            label2id = {l:i for i,l in id2label.items()}
            tokens = {key: torch.row_stack(ls) for key, ls in tokens.items()}
            tokens['labels'] = [label2id[l] for l in L]
            self.model.config.id2label = id2label
            self.model.config.label2id = label2id

            D = DatasetDict({'train': Dataset.from_dict(tokens)})
            for nm, frac in (('holdout',holdout_frac), ('validate',validate_frac)):
                if frac > 0 and frac <=1:
                    D['train'], D[nm] = self.splitter(D['train'], frac)

            self.dataset = D
            if self.task == 'text-generation':
                def f(X):
                    X['labels'] = X['input_ids']
                    return X
                self.dataset = D.map(f)
            return self.dataset

    def train(self, dataset=None, kwargs=dict()):
        if not self.trained:
            if dataset is None:
                dataset = self.dataset

            train_kwargs = {
                'output_dir'                  : self.path_model,
                'evaluation_strategy'         : 'epoch',
                'logging_strategy'            : 'epoch',
                'logging_dir'                 : self.path_model / 'logs',
                'load_best_model_at_end'      : False,
                'save_strategy'               : 'no',
                'learning_rate'               : 1e-5,
                'per_device_train_batch_size' : 32,
                'warmup_steps'                : 600,
                'num_train_epochs'            : 3,
            }
            train_kwargs.update(kwargs)
            train_kwargs['per_device_eval_batch_size']  = train_kwargs['per_device_train_batch_size']

            self.model = self.base.from_config(self.model.config)
            trainer = Trainer(
                model = self.model,
                args = TrainingArguments(**train_kwargs),
                train_dataset = dataset['train'],
                eval_dataset = dataset['validate'],)
            trainer.train()
            self.model = self.model.to('cpu')
            self.model.save_pretrained(self.path_model)
        self.pipe = pipeline(task=self.task, model=self.model, tokenizer=self.tokenizer)

    def classify(self, texts=''):
        def f(text):
            t = self.preprocess(text)['wrapped']
            s = self.model(**t)
            return s.logits.softmax(1).mean(axis=0).detach().numpy()
        d = {'proba': pd.DataFrame([f(text) for text in self._listify(texts)], columns=self.model.config.id2label.values())}
        d['pred'] = d['proba'].idxmax(axis=1).tolist()
        return d

    def tokenize(self, texts):
        outputs = [self.tokenizer(text, truncation=False, padding='max_length', return_tensors='pt') for text in self._listify(texts)]
        return outputs if len(outputs) > 1 else outputs[0]
    
    def wrap_tokens(self, tokens):
        def f(tok):
            T = tok['input_ids'].shape[1]
            sh = (T // self.max_seq_length, self.max_seq_length)
            T = np.product(sh)
            return {key:val[0,-T:].reshape(sh) for key, val in tokens.items()}
        outputs = [f(tok) for tok in self._listify(tokens)]
        return outputs if len(outputs) > 1 else outputs[0]

    def preprocess(self, texts):
        tokens = self.tokenize(texts)
        return {'flat':tokens, 'wrapped':self.wrap_tokens(tokens)}

    def concat_tokens(self, tokens1, tokens2):
        return {key:torch.cat((tokens1[key],tokens2[key]), axis=1) for key in tokens1.keys()}

    def split_tokens(self, tokens, head_length=0, tail_length=0):
        if head_length > 0:
            head = {key:val[:,:head_length] for key,val in tokens.items()}
            tail = {key:val[:,head_length:] for key,val in tokens.items()}
        elif tail_length > 0:
            head = {key:val[:,:-tail_length] for key,val in tokens.items()}
            tail = {key:val[:,-tail_length:] for key,val in tokens.items()}
        else:
            head = tokens
            tail = None
        return head, tail

    def count_words(self, text):
        return len(text.split(" "))

    def generate(self, prompts='', max_new_tokens=20, target_word_count=100, print_steps=True, kwargs=dict()):
        gen_kwargs = {
            'repetition_penalty': 10.0,  # penalty for repeating yourself
            'pad_token_id'      : 50256,
            }
        gen_kwargs.update(kwargs)

        max_length = self.max_seq_length + max_new_tokens
        outputs = []
        for text in self._listify(prompts):
            tokens = self.tokenize(text)
            while self.count_words(text) < target_word_count:
                head, tail = self.split_tokens(tokens, tail_length=self.max_seq_length)
                extended = {'input_ids': self.model.generate(**tail, max_length=max_length, **gen_kwargs)}
                extended['attention_mask'] = torch.ones_like(extended['input_ids'])

                _, new_tokens = self.split_tokens(extended, head_length=self.max_seq_length)
                tokens = self.concat_tokens(tokens, new_tokens)
                new_text = self.tokenizer.decode(new_tokens['input_ids'][0])
                text += new_text
                if print_steps:
                    display(text)
                    print(f'word count = {self.count_words(text)}\n')
            outputs.append(text)
        return outputs if len(outputs) > 1 else outputs[0]
        

root_path = '/content/drive/MyDrive/NLP/tswift/'

def read_google(gid, gsheetid='1BJaEmZF8KIB-zFi4qoP6Detp8ZtgTLG482hEwHKK61I'):
    url = f'https://docs.google.com/spreadsheets/d/{gsheetid}/export?format=csv&gid={gid}'
    return pd.read_csv(url)
    
songs  = read_google(599294473)
mask = songs['genre'].isnull()
if mask.any():
    print(f'There are {mask.sum()} unlabeled rows')
    # rand_genres = np.random.choice(['alternative', 'country', 'pop'], songs.shape[0])
    # songs.loc[mask,'genre'] = rand_genres[mask]
lyrics_lines = read_google(534858430)
lyrics = lyrics_lines.groupby('song_index')['lyric'].agg(lambda x: ' '.join(x))
songs = songs.merge(lyrics, on='song_index').sort_values(['genre', 'release_date', 'song_title'])

mask = songs['genre'].isnull()
holdout_set = songs[mask].copy().reset_index()
songs = songs[~mask].reset_index()
source_data = (songs.groupby('genre')[['lyric']].agg(lambda x: ' '.join(x)).reset_index())

There are 89 unlabeled rows


In [4]:
#Text Classification

max_seq_length = 64
batch_size = 16
num_epochs = 15

model_kwargs = {}
clf = NLP(
    overwrite      = False,
    max_seq_length = max_seq_length,
    base           = AutoModelForSequenceClassification,
    checkpoint     = 'distilbert-base-uncased',
    task           = 'sentiment-analysis',
    path           = root_path,
    path_model     = root_path + f'classifier/{max_seq_length}/',
    kwargs         = model_kwargs)

data_kwargs = {}
clf.make_dataset(
    source_data   = source_data,
    text_col      = 'lyric',
    label_col     = 'genre',
    holdout_frac  = 0.0,
    validate_frac = 0.2,
    )

train_kwargs = {
    'num_train_epochs'            : num_epochs,
    'per_device_train_batch_size' : batch_size,
    'learning_rate'               : 1e-5,
    'warmup_steps'                : 600,
    'save_strategy'               : 'no',
}
clf.train(kwargs=train_kwargs)

df = songs.sample(frac=1).reset_index()
d = clf.classify(df['lyric'].tolist())
df['pred'] = d['pred']




Using existing model at /content/drive/MyDrive/NLP/tswift/classifier/64


In [5]:
display(confusion_matrix(df['genre'],df['pred']))  
print(classification_report(df['genre'],df['pred']))

array([[32,  0,  1],
       [ 1, 61,  2],
       [ 0,  2, 73]])

              precision    recall  f1-score   support

 alternative       0.97      0.97      0.97        33
     country       0.97      0.95      0.96        64
         pop       0.96      0.97      0.97        75

    accuracy                           0.97       172
   macro avg       0.97      0.97      0.97       172
weighted avg       0.97      0.97      0.97       172



In [6]:
clf.classify(holdout_set['lyric'].tolist()) #classify all unreleased 'genre-less' songs

{'pred': ['country',
  'country',
  'country',
  'country',
  'country',
  'pop',
  'country',
  'country',
  'pop',
  'pop',
  'country',
  'country',
  'alternative',
  'pop',
  'country',
  'country',
  'country',
  'country',
  'country',
  'country',
  'alternative',
  'country',
  'country',
  'alternative',
  'country',
  'country',
  'country',
  'country',
  'country',
  'pop',
  'country',
  'country',
  'country',
  'country',
  'pop',
  'country',
  'alternative',
  'country',
  'country',
  'country',
  'pop',
  'pop',
  'country',
  'alternative',
  'alternative',
  'country',
  'alternative',
  'country',
  'alternative',
  'country',
  'country',
  'country',
  'country',
  'country',
  'country',
  'alternative',
  'pop',
  'alternative',
  'country',
  'pop',
  'country',
  'country',
  'country',
  'country',
  'country',
  'country',
  'pop',
  'country',
  'pop',
  'country',
  'country',
  'country',
  'pop',
  'pop',
  'country',
  'alternative',
  'country',
  '

In [7]:
#Text Generators

gens = dict()
for g, df in source_data.groupby('genre'):
    model_kwargs = {}
    gen = NLP(
        overwrite      = False,
        max_seq_length = 64,
        base           = AutoModelForCausalLM, 
        checkpoint     = 'distilgpt2',
        task           = 'text-generation',
        path           = root_path,
        path_model     = root_path + f'{g}/',
        kwargs         = model_kwargs)

    data_kwargs = {}
    gen.make_dataset(
        source_data   = source_data,
        text_col      = 'lyric',
        label_col     = 'genre',
        holdout_frac  = 0.0,
        validate_frac = 0.2,
        )

    train_kwargs = {
        'num_train_epochs'            : 400,
        'per_device_train_batch_size' : 64,
        'learning_rate'               : 1e-5,
        'warmup_steps'                : 600,
        'save_strategy'               : 'no',
    }

    gen.train(kwargs=train_kwargs)
    gens[g] = gen


Using existing model at /content/drive/MyDrive/NLP/tswift/alternative
Using existing model at /content/drive/MyDrive/NLP/tswift/country
Using existing model at /content/drive/MyDrive/NLP/tswift/pop


In [13]:
# Fun Interactive Generator
# Enter a prompt (line 8), choose your genre (line 6), and see what happens

gen_kwargs = {}

gen = gens['pop']

prompt = 'when i think about you'
gen.generate(
    prompt,
    max_new_tokens=20,
    target_word_count=100,
    print_steps=True,
    kwargs=gen_kwargs)

"when i think about you down But all here And your face When it too Now I'm so enough All there was just like"

word count = 24



"when i think about you down But all here And your face When it too Now I'm so enough All there was just like a little street song at myself to look in the reason for something bad tonight Cause he could be with"

word count = 44



"when i think about you down But all here And your face When it too Now I'm so enough All there was just like a little street song at myself to look in the reason for something bad tonight Cause he could be with him You can't see? It's no one who got us girl, do again and coming good"

...

"when i think about you down But all here And your face When it too Now I'm so enough All there was just like a little street song at myself to look in the reason for something bad tonight Cause he could be with him You can't see? It's no one who got us girl, do again and coming good moment Ooh (Stay He won hear she're com only know why say In on our best phone is not think I've been way home And we'll turn down your name Your eyes) The first"

word count = 94



"when i think about you down But all here And your face When it too Now I'm so enough All there was just like a little street song at myself to look in the reason for something bad tonight Cause he could be with him You can't see? It's no one who got us girl, do again and coming good moment Ooh (Stay He won hear she're com only know why say In on our best phone is not think I've been way home And we'll turn down your name Your eyes) The first goes you want that keeps me Oh-ah time of come back together the same world all Don still"

word count = 112



"when i think about you down But all here And your face When it too Now I'm so enough All there was just like a little street song at myself to look in the reason for something bad tonight Cause he could be with him You can't see? It's no one who got us girl, do again and coming good moment Ooh (Stay He won hear she're com only know why say In on our best phone is not think I've been way home And we'll turn down your name Your eyes) The first goes you want that keeps me Oh-ah time of come back together the same world all Don still"

In [14]:
# Classify your new song

d['proba']

,alternative,country,pop
0,0.160816,0.714678,0.124506
1,0.156790,0.826709,0.016501
2,0.177989,0.005639,0.816373
3,0.533410,0.277122,0.189468
4,0.004006,0.005451,0.990543
...,...,...,...
167,0.564811,0.140985,0.294203
168,0.018395,0.967997,0.013608
169,0.122689,0.061269,0.816042
170,0.070545,0.902166,0.027290


In [15]:
# Generators fed holdout songs, then classified

def smart_truncate(content, length=40, suffix=' '):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix

min_words = 500  # stop when this many words are in text
gen_kwargs = {'print_steps'        : False}


gen_song_list = []
true_labels = []
pred_labels = []

for x in holdout_set['lyric']:
    new_songs = dict()
    line = smart_truncate(x)
    for g, gen in gens.items():
        print(g)
        text = line
        for i in range(10):  # run generate up to this many times to try to get min_words
            text = gen.generate(text, **gen_kwargs)  # call generate
            L = len(text.split(" "))  # compute number of words by splitting on spaces
            #if print_while_generating:
            #    display(text)
            #    print()
            if L >= min_words:  # stop if enough words
                break
        new_songs[g] = text
        label = clf.classify(line)['pred']
        gen_song_list.append(text)
        true_labels.append(g)
        pred_labels.append(label)  # classify
        display(text)
        print(f'{g} took {i} iterations to get length {L}: classification {label}\n\n')
        #print(labels)
        #print(new_songs)
    #results = pd.DataFrame(gen_song_list, columns=labels)
    #print(results)

alternative


'He said, "I got me a nice new apartment  It\'s like I can\'t take your mind), we out of she should be waiting on the middle through this is our time Cause that was mad When you and baby why darling You say my daddy rains in an little front They\'re gorgeous it could her forever for people? Baby- what everything And when they never thought there don would let me down (I\'m nothing a feeling, getaway car) I just need to come back friend The way home gonna make us), have But we\'d be here you can\'t know so long oh Oh who\'s not at night You and take their sick It was'

alternative took 9 iterations to get length 115: classification ['pop']


country


'He said, "I got me a nice new apartment  day like that I say he knows Cause you just want to see" But there\'s still over never go down He\'ll be an best if we\'m beautiful of this ain\'t have the very mind And they let it goes Baby-ah Don blood Only So oh time was\'ve been then water will get me in a man known home dress so many away All my own night (Oh, ha A!) again When you\'re not his front car I know what she did that fly It say Now now Ooh) But There\'s who do don\'t see baby one up with them is look at this love'

country took 9 iterations to get length 114: classification ['pop']


pop


...

alternative took 9 iterations to get length 115: classification ['alternative']


country


"I heard a song tonight on the radio  not sorry I let you why's our whoa The only one? You know what all there was too for me when we're so long (You got no) And he knows That could read your name with my time Cause they'll take, then So will be alright But if it won't come back to call It feel sick of us down room baby In a London look at again We can have the dress still meet you smile I don concerned They didn see right now boyI'm those who make me that we're not big world And let some only one are (Oh-ah? Ooh"

country took 9 iterations to get length 112: classification ['alternative']


pop


'I heard a song tonight on the radio  Don\'t tell me now I was really got your head You need to say so long in you want And he make this who\'s finally they\'ll be messed with their So still" (Ah) But there out Well, ah time When we should know that keeps him all right around and only thing can feel okay if fall knows She\'d call nobody had a smile when old sparks girl\'ve been an don\'t see me I\'m the best one place enough of style through everything ready baby Baby-oh Odi to make it\'s really think you said oh You mean It feels like this way In here Don'

pop took 9 iterations to get length 114: classification ['alternative']




In [16]:
display(confusion_matrix(true_labels, pred_labels))
print(classification_report(true_labels, pred_labels))

array([[55, 20, 14],
       [55, 20, 14],
       [55, 20, 14]])

              precision    recall  f1-score   support

 alternative       0.33      0.62      0.43        89
     country       0.33      0.22      0.27        89
         pop       0.33      0.16      0.21        89

    accuracy                           0.33       267
   macro avg       0.33      0.33      0.31       267
weighted avg       0.33      0.33      0.31       267

